## Option Portfolios
Equities have a very straightforward exposure to idiosyncratic and systematic risk. Options, on the other hand, have exposure to not only the underlying asset, but also interest rates, time, and volatility. These exposures are inputs to the Black-Scholes option pricing model. 

$$\ C(S,t)=S_{{t}}N(d_{{1}})-Ke^{{-r(T-t)}}N(d_{{2}}) $$
$$ \ P(S,t)=Ke^{{-r(T-t)}}N(-d_{{2}})-S_{{t}}N(-d_{{1}}) $$

$$\ d_{{1}}={\frac  {\ln {\frac  {S_{{t}}}{K}}+\left(r+{\frac  {1}{2}}\sigma ^{{2}}\right)(T-t)}{\sigma {\sqrt  {T-t}}}};\quad d_{{2}}=d_{{1}}-\sigma {\sqrt  {T-t}}$$

Since these inputs affect the value of the option in question, the partial derivative of the function can tell us how the option value changes when one of these exposures changes holding the others constant. Let’s first discuss all of the partial derivatives of this option pricing model, then code an algorithmic hedging system using `python`.

### The Greeks
Using a Taylor series expansion we can derive all of the greeks. The greeks tell us how we can expect an option or portfolio of options to change when a change occurs in one or more of the option exposures. Something important to note is that all first-order approximations are linear, and the option pricing function is non-linear. This means the more the underlying parameter deviates from the initial partial-derivative calculation the less accurate it will be. This is why the greeks are updated, generally, in real-time, so we can constantly have a new set of expectations for option or portfolio value when something changes.

**Note:** In the following scenarios we are generally considering calls and puts.

#### Delta
The first-order partial-derivative with respect to the underlying asset of the Black-Scholes equation is known as delta. Delta refers to how the option value changes when there is a change in the underlying asset price. Multiplying delta by a +-$1 change in the underlying asset, holding all other parameters constant, will give you the new value of the option. Delta will be positive for long call and short put positions, negative for short call and long put positions.

$$\Delta = \frac{\partial C}{\partial S} = N(d_1)$$

$$\Delta = \frac{\partial P}{\partial S} = -N(-d_1)$$

The (absolute value of) $\Delta$ is close to, but not identical with, the percent moneyness of an option, i.e., the implied probability that the option will expire in-the-money (if the market moves under Brownian motion in the risk-neutral measure). For this reason some option traders use the absolute value of delta as an approximation for percent moneyness. For example, if an out-of-the-money call option has a delta of 0.15, the trader might estimate that the option has approximately a 15% chance of expiring in-the-money. Similarly, if a put contract has a delta of −0.25, the trader might expect the option to have a 25% probability of expiring in-the-money. At-the-money calls and puts have a delta of approximately 0.5 and −0.5 respectively with a slight bias towards higher deltas for ATM calls. The actual probability of an option finishing in the money is its dual delta, which is the first derivative of option price with respect to strike.

#### Gamma
The second-order partial-derivative with respect to the underlying asset of the Black-Scholes equation is known as gamma. Gamma refers to how the option’s delta changes when there is a change in the underlying asset price. Multiplying gamma by a +-$1 change in the underlying asset, holding all other parameters constant, will give you the new value of the option’s delta. Essentially, gamma is telling us the rate of change of delta given a +-1 change in the underlying asset price. Gamma is always positive for long positions and negative for short positions.

$$\Gamma = \frac{\partial^2 C}{\partial S^2} = \frac{\partial^2 P}{\partial S^2} = \frac{1}{S\sigma\sqrt{T-t}}\frac{1}{\sqrt{2\pi}}e^{-\frac{d_1^2}{2}} $$

#### Vega
The first-order partial-derivative with respect to the underlying asset volatility of the Black-Scholes equation is known as $\textrm{Vega}$. $\textrm{Vega}$ refers to how the option value changes when there is a change in the underlying asset volatility. Multiplying $\textrm{Vega}$ by a +-1% change in the underlying asset volatility, holding all other parameters constant, will give you the new value of the option. $\textrm{Vega}$ will be positive for long positions and negative for short positions.

$$ \textrm{Vega} = \frac{\partial C}{\partial\sigma} = \frac{\partial P}{\partial\sigma} = S\sqrt{T-t}\frac{1}{\sqrt{2\pi}}e^{-\frac{d_1^2}{2}}$$

#### Theta
The first-order partial-derivative with respect to the time until option expiration of the Black-Scholes equation is known as $\Theta$. $\Theta$ refers to how the option value changes as time passes. Generally annualized, theta refers to the change in the value of an option when a single day passes. $\Theta$ will be positive for short positions and negative for long positions.

$$ \Theta = \frac{\partial C}{\partial t} = - K e^{-r(T-T)}rN(d_1 - \sigma\sqrt{T-t})-S\frac{\sigma}{2\sqrt{T-t}}\frac{1}{\sqrt{2\pi}}e^{-\frac{d_1^2}{2}}$$

$$ \Theta = \frac{\partial P}{\partial t} = K e^{-r(T-T)}rN(-d_1 + \sigma\sqrt{T-t})-S\frac{\sigma}{2\sqrt{T-t}}\frac{1}{\sqrt{2\pi}}e^{-\frac{d_1^2}{2}}$$

#### Rho
The first-order partial-derivative with respect to the risk-free rate of the Black-Scholes equation is known as $\rho$. $\rho$ refers to how the option value changes as the interest rate changes. Multiplying $\rho$ by a +-1% change in the interest rate, holding all other parameters constant, will give you the new value of the option. $\rho$ will be positive for long call and short put positions and negative for short call and long put positions.

$$\rho = \frac{\partial C}{\partial\rho} = Ke^{-r(T-t)}(T-t)N(d_1-\sigma\sqrt{T-t})$$

$$\rho = \frac{\partial P}{\partial\rho} = -Ke^{-r(T-t)}(T-t)N(-d_1+\sigma\sqrt{T-t})$$

## Dynamic Hedging

Consider the following case:

a colleague currently has a short position in 1000 NVDA calls, she wants to hedge her exposure to changes in volatility, movements in the underlying asset, and the speed of movements in the underlying asset. You’re on the risk-management desk and offer to construct a dynamic hedge to be rebalanced daily. How can we neutralize her exposure to the option’s $\textrm{Vega}$, $\Delta$, and $\Gamma$ ?

We can use the first week of linear algebra to help us construct a solution.

The first thing to realize is that to neutralize exposure to greeks we are going to need offsetting positions in other options. There are three greeks to neutralize, so we need three options to create three equations of greeks and weights with three unknowns (the weights in the other tradable options). However, the trick here is realizing that the partial derivative of the underlying asset with respect to itself is just 1, this means the underlying asset has $\Delta=1$ and all other greek values are 0. This means we can construct a portfolio of two tradable options, find appropriate weights to neutralize the greeks, then take an offsetting position in the underlying asset — effectively neutralizing exposure to all three greeks.

### Python for Dynamic Hedging
Consider the following code that models European calls and puts$\ldots$

In [5]:
import math
from scipy.stats import norm

class EuropeanCall:

    def __init__(self, S, sigma, K, ttm, r):
        self.S = S
        self.sigma = sigma
        self.K = K
        self.ttm = ttm
        self.r = r
        self.price = self.call_price()
        self.delta = self.call_delta()
        self.gamma = self.call_gamma()
        self.vega = self.call_vega()
		
    def d(self):
        b = math.exp(-self.r*self.ttm)
        d1 = math.log(self.S/(b*self.K)) + .5*(self.sigma**2)*self.ttm
        d1 = d1/(self.sigma*(self.ttm**.5))
        return d1

    def call_delta(self):
        d1 = self.d()
        z1 = norm.cdf(d1)
        return z1

    def call_gamma(self):
        d1 = self.d()
        z1 = norm.cdf(d1)
        z2 = z1/(self.S*self.sigma*math.sqrt(self.ttm))
        return z2

    def call_vega(self):
        d1 = self.d()
        z1 = norm.cdf(d1)
        z2 = self.S*z1*math.sqrt(self.ttm)
        return z2/100

    def call_price(self):
        d1 = self.d()
        z1 = norm.cdf(d1)
        z1 = z1*self.S
        d2 = self.d() - self.sigma*math.sqrt(self.ttm)
        z2 = norm.cdf(d2)
        z2 = math.exp(-self.r*self.ttm)*self.K*z2
        return z1 - z2

# class EuropeanPut:

#     def put_delta(
#         self, asset_price, asset_volatility, strike_price,
#         time_to_expiration, risk_free_rate
#             ):
#         b = math.exp(-risk_free_rate*time_to_expiration)
#         x1 = math.log(asset_price/(b*strike_price)) + .5*(asset_volatility*asset_volatility)*time_to_expiration
#         x1 = x1/(asset_volatility*(time_to_expiration**.5))
#         z1 = norm.cdf(x1)
#         return z1 - 1

#     def put_gamma(
#         self, asset_price, asset_volatility, strike_price,
#         time_to_expiration, risk_free_rate
#             ):
#         b = math.exp(-risk_free_rate*time_to_expiration)
#         x1 = math.log(asset_price/(b*strike_price)) + .5*(asset_volatility*asset_volatility)*time_to_expiration
#         x1 = x1/(asset_volatility*(time_to_expiration**.5))
#         z1 = norm.cdf(x1)
#         z2 = z1/(asset_price*asset_volatility*math.sqrt(time_to_expiration))
#         return z2

#     def put_vega(
#         self, asset_price, asset_volatility, strike_price,
#         time_to_expiration, risk_free_rate
#             ):
#         b = math.exp(-risk_free_rate*time_to_expiration)
#         x1 = math.log(asset_price/(b*strike_price)) + .5*(asset_volatility*asset_volatility)*time_to_expiration
#         x1 = x1/(asset_volatility*(time_to_expiration**.5))
#         z1 = norm.cdf(x1)
#         z2 = asset_price*z1*math.sqrt(time_to_expiration)
#         return z2/100

#     def put_price(
#         self, asset_price, asset_volatility, strike_price,
#         time_to_expiration, risk_free_rate
#             ):
#         b = math.exp(-risk_free_rate*time_to_expiration)
#         x1 = math.log((b*strike_price)/asset_price) + .5*(asset_volatility*asset_volatility)*time_to_expiration
#         x1 = x1/(asset_volatility*(time_to_expiration**.5))
#         z1 = norm.cdf(x1)
#         z1 = b*strike_price*z1
#         x2 = math.log((b*strike_price)/asset_price) - .5*(asset_volatility*asset_volatility)*time_to_expiration
#         x2 = x2/(asset_volatility*(time_to_expiration**.5))
#         z2 = norm.cdf(x2)
#         z2 = asset_price*z2
#         return z1 - z2

#     def __init__(
#         self, asset_price, asset_volatility, strike_price,
#         time_to_expiration, risk_free_rate
#             ):
#         self.asset_price = asset_price
#         self.asset_volatility = asset_volatility
#         self.strike_price = strike_price
#         self.time_to_expiration = time_to_expiration
#         self.risk_free_rate = risk_free_rate
#         self.price = self.put_price(asset_price, asset_volatility, strike_price, time_to_expiration, risk_free_rate)
#         self.delta = self.put_delta(asset_price, asset_volatility, strike_price, time_to_expiration, risk_free_rate)
#         self.gamma = self.put_gamma(asset_price, asset_volatility, strike_price, time_to_expiration, risk_free_rate)
#         self.vega = self.put_vega(asset_price, asset_volatility, strike_price, time_to_expiration, risk_free_rate)


For the case above consider the following conditions$\ldots$

* short 1000 NVDA calls@545
* NVDA price $543
* NVDA implied volatility 53%
* 1 month until option expiration
* 30 day LIBOR rate 1.5%

Using these inputs we can find the theoretical value of our colleague’s option position…

In [7]:
short_nvda = EuropeanCall(543, .53, 545, 30/365, .015)
print("{:.2f}".format(short_nvda.price*(1000)))

32264.05


This means our colleague would have collected a $32264.05 premium for the sale of the options.

What about the greeks ? To find our colleague’s position in the greeks we can just print them and multiply by the position…

In [9]:
print("{:.2f}".format(short_nvda.delta*(-1000)))
print("{:.2f}".format(short_nvda.gamma*(-1000)))
print("{:.2f}".format(short_nvda.vega*(-1000)))

-523.88
-6.35
-815.54


This means the net portfolio $\Delta$, $\Gamma$ and $\textrm{Vega}$ are$\ldots$

* Delta: -523.88
* Gamma: -6.35
* Vega: -815.54

Now let’s consider two more tradable call options, all the other parameters are the same but the two strike prices are 550 and 555$\ldots$

In [11]:
call_a = EuropeanCall(543, .53, 550, 30/365, .015)
call_b = EuropeanCall(543, .53, 555, 30/365, .015)

print("{:.4f}".format(call_a.delta))
print("{:.4f}".format(call_a.gamma))
print("{:.4f}".format(call_a.vega))

print("{:.4f}".format(call_b.delta))
print("{:.4f}".format(call_b.gamma))
print("{:.4f}".format(call_b.vega))

0.4999
0.0061
0.7782
0.4762
0.0058
0.7413


#### Call Option A:

* Delta: 0.49991
* Gamma: 0.00605
* Vega: 0.77823

#### Call Option B:

* Delta: 0.47616
* Gamma: 0.00577
* Vega: 0.74126

#### Underlying Asset (NVDA Stock):

* Delta: 1
* Gamma: 0
* Vega: 0

Using a combination of these assets we can neutralize our portfolio’s exposure to $\Delta$, $\Gamma$ and $\textrm{Vega}$. The question is how ? The answer: linear algebra.

### Greek Neutralization
The greeks we are interested in neutralizing in the current portfolio can be expressed as a vector…

$$\begin{bmatrix}
\Delta\\
\Gamma \\
\textrm{Vega}
\end{bmatrix}=
\begin{bmatrix}
-523.88\\
-6.35 \\
-815.54
\end{bmatrix}$$

The goal is to find the weights of the three assets we are capable of trading to neutralize these values. First, we will look to neutralize $\Gamma$ and $\textrm{Vega}$, then using the underlying asset, we will neutralize $\Delta$.

$$
\begin{bmatrix}
.00605 & .00577 \\
.77823 & .74126 \end{bmatrix}
\begin{bmatrix}
w_1 \\
w_2 \end{bmatrix} = 
\begin{bmatrix}
6.35 \\
815.54 \end{bmatrix}$$

This means by inverting the matrix containing the greek values for the tradable options we can find the appropriate weights…

$$
\begin{bmatrix}
w_1 \\
w_2 \end{bmatrix} =
\begin{bmatrix}
.00605 & .00577 \\
.77823 & .74126 \end{bmatrix}^{-1}
\begin{bmatrix}
6.35 \\
815.54 \end{bmatrix}$$

We can do this using `python`$\ldots$

**Note:** Though we have a short position in the original option we multiply by a positive 1000 as we want to invert the matrix to a positive $\Gamma$, $\textrm{Vega}$ position.

In [44]:
import numpy as np

greeks = np.array([[call_a.gamma, call_b.gamma], [call_a.vega, call_b.vega]])
portfolio_greeks = [[short_nvda.gamma*1000], [short_nvda.vega*1000]]

# We need to round otherwise we can end up with a non-invertible matrix
#inv = np.linalg.inv(np.round(greeks, 2))
inv = np.linalg.pinv(greeks)
print(inv)

[[0.00524506 0.67368158]
 [0.00499592 0.64168088]]


In [40]:
inv = np.linalg.inv(np.round(greeks, 2))
print (inv)

[[-1850.    25.]
 [ 1950.   -25.]]


We have effectively found the inverse of the matrix, the dot product will be the resulting weights for both tradable options$\ldots$


In [45]:
w = np.dot(inv, portfolio_greeks)
print(w)

[[549.44708874]
 [523.34768136]]


Using these weights we will have effectively neutralized our exposure to gamma and vega…


In [46]:
#print(np.round(np.dot(np.round(greeks, 4), w) - portfolio_greeks))
#print(np.dot(np.round(greeks, 2), w) - portfolio_greeks)
print(np.dot(greeks, w) - portfolio_greeks)

[[5.32907052e-15]
 [4.54747351e-13]]


Now that the exposure to $\Gamma$ and $\textrm{Vega}$ is neutralized we need to neutralize our new exposure to $\Delta$. To find our new exposure, we take the sum-product of all option positions in our portfolio with their respective deltas$\ldots$


In [47]:
# Greeks including delta
portfolio_greeks = [[short_nvda.delta*-1000], [short_nvda.gamma*-1000], [short_nvda.vega*-1000]]
greeks = np.array([[call_a.delta, call_b.delta], [call_a.gamma, call_b.gamma], [call_a.vega, call_b.vega]])
#print(np.round(np.dot(np.round(greeks, 4), w) + portfolio_greeks))
#print((np.dot(np.round(greeks, 2), w) + portfolio_greeks))
print((np.dot(greeks, w) + portfolio_greeks))

[[3.41060513e-13]
 [5.32907052e-15]
 [4.54747351e-13]]


In [52]:
greeks = np.array([[call_a.delta, call_b.delta, 1], [call_a.gamma, call_b.gamma, 0], [call_a.vega, call_b.vega, 0]])
portfolio_greeks = [[short_nvda.delta*1000], [short_nvda.gamma*1000], [short_nvda.vega*1000]]

inv = np.linalg.inv(np.round(greeks, 4))
print (inv)

[[ 0.00000000e+00  8.85663082e+04 -6.92951016e+02]
 [-0.00000000e+00 -9.29749104e+04  7.28793309e+02]
 [ 1.00000000e+00  3.54838710e-01 -6.45161290e-01]]


In [53]:
np.dot(inv, portfolio_greeks)

array([[-2.77513752e+03],
       [ 4.01342410e+03],
       [-2.24765325e-02]])

After multiplying our new options positions by the original greeks we find that our net delta position is -46. This means by purchasing 46 shares of the underlying asset (NVDA) we will have a delta, gamma, and vega neutral portfolio. This means the value of our option portfolio will not change when there are changes in the underlying asset price, underlying asset volatility, or the speed in which the underlying asset price changes. The code laid out herein can be implemented directly into a live trading system.

#### Final $\Delta$, $\Gamma$, and $\textrm{Vega}$ Neutral Portfolio
* -1000 NVDA Calls
* 8641 Call A Options
* -8006 Call B Options
* 46 Shares of NVDA